In [41]:
from honeybee_energy.schedule.day import ScheduleDay
import honeybee_energy.lib.scheduletypelimits as schedule_types
from ladybug.dt import Time, Date
from ladybug.datacollection import HourlyContinuousCollection
from ladybug.analysisperiod import AnalysisPeriod
from ladybug.datatype.generic import GenericType

simple_office = ScheduleDay('Simple Office Occupancy',[0, 1, 0], [Time(0, 0), Time(9, 0), Time(17, 0)])

print(list(simple_office.data_collection().values))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
from honeybee_energy.schedule.day import ScheduleDay
from honeybee_energy.schedule.rule import ScheduleRule
from honeybee_energy.schedule.ruleset import ScheduleRuleset
from honeybee_energy.schedule.typelimit import ScheduleTypeLimit
import honeybee_energy.lib.scheduletypelimits as schedule_types

from ladybug.dt import Date, Time
from ladybug.datatype import fraction
from ladybug.analysisperiod import AnalysisPeriod

weekday_office = ScheduleDay('Weekday Office Occupancy', [0, 1, 0],
                                 [Time(0, 0), Time(9, 0), Time(17, 0)])
saturday_office = ScheduleDay('Saturday Office Occupancy', [0, 0.25, 0],
                                [Time(0, 0), Time(9, 0), Time(17, 0)])
sunday_office = ScheduleDay('Sunday Office Occupancy', [0])
sat_rule = ScheduleRule(saturday_office, apply_saturday=True)
sun_rule = ScheduleRule(sunday_office, apply_sunday=True)
summer_office = ScheduleDay('Summer Office Occupancy', [0, 1, 0.25],
                            [Time(0, 0), Time(6, 0), Time(22, 0)])
winter_office = ScheduleDay('Winter Office Occupancy', [0])
schedule = ScheduleRuleset('Office Occupancy', weekday_office,
                            [sat_rule, sun_rule], schedule_types.fractional,
                            sunday_office, summer_office, winter_office)

schedule.schedule_rules

(ScheduleRule: Saturday Office Occupancy [days applied: saturday] [date range: 01 Jan - 31 Dec],
 ScheduleRule: Sunday Office Occupancy [days applied: sunday] [date range: 01 Jan - 31 Dec])

In [22]:
room.properties.energy.lighting.schedule[0]

ScheduleRule: Hospital BLDG_LIGHT_EXTD_SCH_Sat [days applied: saturday] [date range: 01 Jan - 31 Dec]

In [2]:
print(room.properties.energy.lighting.schedule.summer_designday_schedule)

Schedule:Day:Interval,
 Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn, !- schedule name
 ,                         !- schedule type limits
 No,                       !- interpolate to timestep
 24:00,                    !- time 1 {hh:mm}
 1.0;                      !- value until time 1


In [1]:
from honeybee.model import Model
from honeybee_doe2.properties.activitydescription import DayScheduleDoe, DayScheduleType, WeekScheduleDoe


tmodel = Model.from_file('tests\\assets\\hospcube.hbjson')
room1 = tmodel.rooms[0]


my_sch = WeekScheduleDoe.from_schedule_ruleset(room1.properties.energy.lighting.schedule, stype=DayScheduleType.FRACTION)
my_sch

"Hospital BLDG_LIGHT_EXTD_SCH" = WEEK-SCHEDULE
   TYPE     = FRACTION
     (CDD)  "Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn"
     (HDD)  "Hospital BLDG_LIGHT_EXTD_SCH_WntrDsn"
     (SAT)  "Hospital BLDG_LIGHT_EXTD_SCH_Sat"
     (FRI, WED)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
     (MON, FRI)  "Hospital BLDG_LIGHT_EXTD_SCH_Wkdy"
     (SUN)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
   ..

In [1]:
from honeybee.model import Model
from honeybee_doe2.properties.activitydescription import DayScheduleDoe, DayScheduleType, WeekScheduleDoe


tmodel = Model.from_file('tests\\assets\\hospcube.hbjson')
room1 = tmodel.rooms[0]


my_sch = WeekScheduleDoe.from_schedule_ruleset(room1.properties.energy.lighting.schedule, stype=DayScheduleType.FRACTION)
my_sch

"Hospital BLDG_LIGHT_EXTD_SCH" = WEEK-SCHEDULE
   TYPE     = FRACTION
     (CDD)  "Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn"
     (HDD)  "Hospital BLDG_LIGHT_EXTD_SCH_WntrDsn"
     (SAT)  "Hospital BLDG_LIGHT_EXTD_SCH_Sat"
     (MON, FRI)  "Hospital BLDG_LIGHT_EXTD_SCH_Wkdy"
   ..

In [3]:
from honeybee.model import Model
from dataclasses import dataclass
from honeybee_doe2.properties.activitydescription import DayScheduleDoe, DayScheduleType, Days
from typing import List
from honeybee_doe2.utils.doe_formatters import short_name
from honeybee_energy.schedule.ruleset import ScheduleRuleset

@dataclass
class WeekScheduleDoe:
    name: str = None
    stype: DayScheduleType = None
    values: List = None

    @classmethod
    def from_schedule_ruleset(
            cls, schedule_ruleset: ScheduleRuleset, stype: DayScheduleType):
        """ Create a doe2 Week Schedule from a honeybee ScheduleRuleset."""
        days_of_the_week = [Days.SUN.value, Days.MON.value, Days.TUE.value,
                            Days.WED.value, Days.THUR.value, Days.FRI.value, Days.SAT.value]
        ruleset_daylib = []

        name = schedule_ruleset.display_name
        stype = stype

        values = []

        if schedule_ruleset.summer_designday_schedule:
            values.append(
                [[Days.CDD.value],
                 f'"{schedule_ruleset.summer_designday_schedule.display_name}"'])
        if schedule_ruleset.winter_designday_schedule:
            values.append(
                [[Days.HDD.value],
                 f'"{schedule_ruleset.winter_designday_schedule.display_name}"'])

        for rule in schedule_ruleset.schedule_rules:
            sch_days = []

            if 'saturday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.SAT.value)
                ruleset_daylib.append(Days.SAT.value)
            if 'monday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.MON.value)
                ruleset_daylib.append(Days.MON.value)
            if 'tuesday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.TUE.value)
                ruleset_daylib.append(Days.TUE.value)
            if 'wednesday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.WED.value)
                ruleset_daylib.append(Days.WED.value)
            if 'thursday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.THUR.value)
                ruleset_daylib.append(Days.THUR.value)
            if 'friday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.FRI.value)
                ruleset_daylib.append(Days.FRI.value)
            if 'sunday'.upper() in (obj.upper() for obj in rule.days_applied):
                sch_days.append(Days.SUN.value)
                ruleset_daylib.append(Days.SUN.value)

            if len(sch_days) >= 2:
                values.append([[sch_days[0], sch_days[-1]],
                               f'"{rule.schedule_day.display_name}"'])
            elif len(sch_days) == 1:
                values.append([sch_days, f'"{rule.schedule_day.display_name}"'])

            default_days = [
                day for day in sorted(days_of_the_week)
                if day not in sorted(ruleset_daylib)]
            
            # if len(default_days) >= 2:
            #     values.append(
            #         [[default_days[0],
            #           default_days[-1]],
            #          f'"{schedule_ruleset.default_day_schedule.display_name}"'])
            if len(default_days) == 1:
                values.append(
                    [default_days,
                     f'"{schedule_ruleset.default_day_schedule.display_name}"'])

        return cls(name=name, stype=stype, values=values)

    def to_inp(self):

        obj_lines = []
        obj_lines.append(f'"{self.name}" = WEEK-SCHEDULE')
        obj_lines.append(f'\n   TYPE     = {self.stype.value}')
        for obj in self.values:
            objstr = ', '.join([str(val) for val in obj[0]])
            obj_lines.append(f'\n     ({objstr})  {obj[1]}')
        obj_lines.append(f'\n   ..')

        return ''.join([line for line in obj_lines])

    def __repr__(self):
        return self.to_inp()




tmodel = Model.from_file('tests\\assets\\hospcube.hbjson')
room1 = tmodel.rooms[0]


my_sch = WeekScheduleDoe.from_schedule_ruleset(room1.properties.energy.lighting.schedule, stype=DayScheduleType.FRACTION)
my_sch

"Hospital BLDG_LIGHT_EXTD_SCH" = WEEK-SCHEDULE
   TYPE     = FRACTION
     (CDD)  "Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn"
     (HDD)  "Hospital BLDG_LIGHT_EXTD_SCH_WntrDsn"
     (SAT)  "Hospital BLDG_LIGHT_EXTD_SCH_Sat"
     (MON, FRI)  "Hospital BLDG_LIGHT_EXTD_SCH_Wkdy"
     (SUN)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
   ..

In [ ]:

tmodel = Model.from_file('tests\\assets\\hospcube.hbjson')
room1 = tmodel.rooms[0]

